In [1]:
import pandas as pd
import pickle
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from src.llm_engine.gemini_agent import Agent

#agent = ChatAgent()

In [2]:
df = pd.read_csv(r"../data/GridPuzzle_processed.csv")

df.head()

,key,id,question,answer,Mistral-7b,Llama-13b,gemini-pro,gpt-4-turbo,Claude-3,actual_question,Clues,categories,number_of_clues,number_categories,category_names
0,341.json,802,Nicholas and Edith are having a small dinner p...,1984 | Annata Branco | gewurztraminer\n1988 | ...,| 1984 | Annata Branco | gewurztraminer |\n |-...,Let's solve the puzzle step by step.\n \n Step...,"Step-by-step solution:\n 1. From clue 6, we kn...","Step-by-step solution:\n \n 1. From clue 6, we...","Step-by-step solution:\n \n 1. From clue 6, we...",Nicholas and Edith are having a small dinner p...,Clues:\n1. The Ece Suss was bottled sometime a...,"vintages : 1984, 1988, 1992, 1996.\nwines : An...",6,3,vintages wines types
1,341.json,726,The world-famous Punta Santiago Rose Gardening...,first | bon silene | Renee\nsecond | perle d'o...,| first | calocarpa | Hannah\n |---------|----...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n 1. From clue 1, we kn...","Step-by-step solution:\n \n 1. From clue 3, we...","Step-by-step solution:\n \n 1. From clue 1, we...",The world-famous Punta Santiago Rose Gardening...,Clues:\n1. The rose that won fourth place was ...,"placements : first, second, third, fourth.\nro...",5,3,placements roses gardeners
2,341.json,16,A standardized test has recently been develope...,55% | Archimedes | CIT\n62% | Galen | UCLA\n69...,| 55% | Zeno | CIT |\n |------|------|-----|\n...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n 1. From clue 2, we kn...","Step-by-step solution:\n \n 1. From clue 4, Pr...","Step-by-step solution:\n \n 1. From clue 4, we...",A standardized test has recently been develope...,Clues:\n1. Zeno finished with a score that was...,"scores : 55%, 62%, 69%, 76%.\na.i. : Archimede...",5,2,scores institutions
3,341.json,698,Gil Glabbers has lived on the coast of Guernse...,1962 | Tracy | Cherbourg\n1969 | Felipe | Bour...,| 1962 | Felipe (Bournemouth) |\n | 1969 | Kar...,Let's break down the clues step by step:\n \n ...,"Step-by-step solution:\n \n 1. From clue 2, we...","Step-by-step solution:\n \n 1. From Clue 2, Tr...","Step-by-step solution:\n \n 1. From clue 2, we...",Gil Glabbers has lived on the coast of Guernse...,Clues:\n1. The message from Plymouth was sent ...,"year sent : 1962, 1969, 1976, 1983.\nwriters :...",5,3,sent writers origins
4,341.json,444,Jamie never misses an opportunity to watch the...,2 | April 22 | Eastbrook\n9 | April 10 | Islet...,| Dates | Locations |\n |---|---|\n | April 10...,Let's break down the clues and solve the puzzl...,"Step-by-step solution:\n \n 1. From clue 1, we...",Step-by-step solution:\n \n Step 1: Analyze th...,"Step-by-step solution:\n \n 1. From clue 3, we...",Jamie never misses an opportunity to watch the...,Clues:\n1. The outing to Eastbrook logged 7 fe...,"shooting stars : 2, 9, 16, 23.\ndates : April ...",4,3,stars dates locations


In [104]:
from pprint import pprint
pprint(df["Clues"].iloc[4] + "\n"+ df["categories"].iloc[4])

('Clues:\n'
 '1. The outing to Eastbrook logged 7 fewer shooting stars than the trip to '
 'Isleton.\n'
 '2. The April 17 outing was in Gilmore City.\n'
 '3. The outing where they saw 16 shooting stars was in Gilmore City.\n'
 '4. The April 10 outing logged 7 more shooting stars than the April 22 '
 'outing.\n'
 'shooting stars : 2, 9, 16, 23.\n'
 'dates : April 7, April 10, April 17, April 22.\n'
 'locations : Eastbrook, Gilmore City, Isleton, Manchester.')


In [19]:
agent_name = "llm_agent"
system_prompt = """You are a logic grid puzzle solving assistant that produces clear deductive steps. When solving puzzles:\n\n1. Skip repeating the original clues - assume they're already understood.\n\n2. Start immediately with meaningful deductions derived from combining and analyzing clues.\n\n3. For each numbered deduction step:\n   - Present only the NEW inference or conclusion reached\n   - Briefly explain which constraints led to this conclusion\n   - Show any concrete mappings or assignments that result (X = Y, A cannot be B, etc.)\n\n4. Include specific variable assignments and eliminations in your steps (e.g., \"Wine A must be from 1988\" or \"Person B cannot have the red car\").\n\n5. Focus on recording the mapping progression - which variables have been definitively assigned and which possibilities have been eliminated.\n\n6. Document crucial decision points and branching logic when necessary.\n\n7. Each step must contribute new information toward the solution without repeating previous deductions.\n\n8. When appropriate, summarize the current state of the solution grid after significant deductions.\n\nYour output should read like the step-by-step solution path in a puzzle solver's notebook, and do not return any solution directly"""

agent = Agent(agent_name=agent_name, system_prompt=system_prompt)

In [22]:
clues = df["Clues"].iloc[0]
categories = df["categories"].iloc[0]

user_query = clues + "\n" + categories

llm_agent_response = agent.perform_action(user_query)

In [23]:
from pprint import pprint
print(llm_agent_response)

1.  From clues 1 and 5, the bottling order is Annata Branco -> Ece Suss -> Bianca Flaux.

2.  Clue 2: Bianca Flaux was bottled 4 years before Vendemmia.

3.  Clue 6: 1984 = gewurztraminer.

4.  Clue 3: 1988 = pinot noir.

5.  Combining clue 2 and the vintages, Bianca Flaux and Vendemmia must be either 1984/1988 or 1992/1996. Since 1984 and 1988 are taken:
    *   Bianca Flaux = 1992
    *   Vendemmia = 1996

6.  From step 1 and 5, the order is Annata Branco -> Ece Suss -> Bianca Flaux (1992). Therefore:
    *   Ece Suss cannot be 1996.
    *   Annata Branco cannot be 1992 or 1996.

7.  Since Bianca Flaux = 1992 and Vendemmia = 1996, and the order is Annata Branco -> Ece Suss -> Bianca Flaux, then:
    *   Annata Branco = 1984
    *   Ece Suss = 1988

8.  Combining steps 3 and 7:
    *   Annata Branco (1984) = gewurztraminer

9.  Combining steps 4 and 7:
    *   Ece Suss (1988) = pinot noir

10. Clue 4: merlot is either Annata Branco or Bianca Flaux. Since Annata Branco is gewurztramine

In [98]:
print(df["answer"].iloc[10])

January | Rosie | shih tzu
February | Lydia | black lab
March | Kayla | schnauzer
April | Violet | boxer


In [24]:
from itertools import combinations

def initialize_probability_tables(num_categories, categories, cat_topics):
    """
    Initialize uniform probability matrices for all pairs of categories.
    
    Parameters:
    num_categories: int - Number of categories
    categories: list - List of category strings with comma-separated subcategories
    
    Returns:
    matrices: dict - Maps category index tuples to probability matrices
    category_mapping: dict - Maps topic strings to category index tuples
    
    Example:
    >>> matrices, mapping = initialize_probability_tables(3, ["red, blue", "small, large", "cheap, expensive"])
    """
    r = 2
    combinations_ls = list(combinations(np.arange(num_categories), r))
    matrices = {}
    for index, combination in enumerate(combinations_ls):
        c1 , c2 = categories[combination[0]].split(", "), categories[combination[1]].split(", ")
       
        matrix =  np.full((len(c1), len(c2)), 1 / max(len(c1), len(c2)))
        topic = cat_topics[combination[0]] + "x" + cat_topics[combination[1]]
        matrices[topic] = matrix
       
    return matrices

In [26]:
import re
import json
cat_topics = []
categories = []
cat_str = df["categories"].iloc[0].split("\n")
for item in cat_str:
    cat_topics.append(item.split(" : ")[0])
    categories.append(item.split(" : ")[1])

print(type(cat_topics), cat_topics)

probability_matrices = initialize_probability_tables(len(categories), categories, cat_topics) 


<class 'list'> ['vintages', 'wines', 'types']


In [12]:
from itertools import combinations
def populate_categories(categories, index):
    
    cat_topics = []
    categories = []

    cat_str = df["categories"].iloc[index].split("\n")
    for item in cat_str:
        cat_topics.append(item.split(" : ")[0])
        categories.append(item.split(" : ")[1].split(", "))

    examples = {}
    r = 2
    combinations_ls = list(combinations(np.arange(len(cat_topics)), r))

    for topic_cob in combinations_ls:
        values = []
        key_1, key_2 = topic_cob
        for i in range(len(categories[0])):
            for j in range(len(categories[0])):

                values.append((categories[key_1][i], categories[key_2][j]))

        examples[cat_topics[topic_cob[0]] + "__x__" + cat_topics[topic_cob[1]]] = values

    return examples

In [13]:
index=4
examples = populate_categories(categories=categories, index=index)



In [27]:
system_prompt = f"""You are a logical reasoning agent.

                    Given a complex sequential list of logical statements about n-categories and corresponding n-2D matrices depicting inter-category
                    item mappings, systematically deconstruct each logical deduction. Critically evaluate the relational insights, translating these
                    abstract logical statements into quantitative probability weights for specific matrix cells. 
                    The weighting process should be nuanced, reflecting the contextual strength, coherence, and inferential power of each logical
                    statement as it relates to the underlying categorical relationships.
                    
                    Comprehending Deductions & Updating Probabilities
                        
                        Carefully process each logical deduction, ensuring clarity by merging sequential steps when appropriate.
                        Use conclusive deductions to:

                            Identify the relevant probability matrix that requires updating.
                            Determine the exact cells to modify, assigning new weights based on the strength of the deduction.

                    Dynamic Adjustments

                        Continuously reassess all decisions as new information becomes available.
                        If new deductions conflict with prior reasoning, promptly revise the probability matrix or discard incorrect conclusions.
                        Ensure that all updates reflect the most accurate and up-to-date analysis.

                    Use these probability values if the information you find fall into one of the categories:
                        
                        - Definite: 1.0
                        - Eliminated: 0.0
                        - Uncertain: 0.5
                        - Most Likely: 0.6-0.9
                    
                    Remember, as with all grid-based logic puzzles, no option in any category will ever be used more than once.
                    Your task is to maintain precision and adaptability, ensuring logical consistency in every update.
                """

agent_job = f"""Process the given  resources below and give the output as expected in the format below.
                
                Available Resources:
                        'Logical Step-By-Step Deductions': {llm_agent_response},
                        'Initial Probability Matrices': {probability_matrices},
                        'Category Mapping': {df["categories"].iloc[3]}

                - You should use the format below

                    *** ("matrix_name", probability_value, row_index, column_index)

                Return the sequence of probability updates
            """

In [14]:
system_prompt = f"""Logical Reasoning Mapping Framework

                    Input: 
                    - Set of logical deductions and it's associated reasoning
                    - Item-item pairs
                    - Category Pair Names

                    Objective:
                    Assign probabilistic weights to item-item mappings based on:
                    1. Reasoning clarity
                    2. Evidential strength
                    3. Logical coherence
                    4. Contextual relevance

                    Mapping Probability Scale:
                    - 1.0: Absolute Certainty (Unequivocal logical proof)
                    - 0.75-0.9: Moderate Confidence (Solid but Indirect Reasoning)
                    - 0.4-0.5: Low Confidence (Weak or Circumstantial Reasoning)
                    - 0.1-0.35: Minimal Confidence (Speculative Reasoning)
                    - 0.0: No Confidence (No Viable Logical Connection)

                    Evaluation Criteria:
                    - Direct Logical Chain: Maximize probability
                    - Indirect Inference: Moderate probability
                    - Circumstantial Evidence: Reduce probability
                    - Contradictory Evidence: Zero probability
                    
                    Ultimate Goal:
                        - Compare each deduction and item-item pair and based on its relevance and reasoning assign probability weights.
                        - Remember, as with all grid-based logic puzzles, no item in any pair will ever be used more than once. 
                                                       
                """

agent_job = f"""Process the given  resources below and give the output as expected in the format below.
                
                Available Resources:
                        'Logical Step-By-Step Deductions': {llm_agent_response},
                        'Item-Item Mapping': {examples},
                        'Topic Pairs': {list(examples.keys())}

                Remember, as with all grid-based logic puzzles, no item in any pair will ever be used more than once. 
                Return the assigned weights for item-item mapping
            """

In [28]:
from pprint import pprint

import json


prob_agent = agent = Agent(agent_name="prob_agent_1", system_prompt=system_prompt, json=True)


prob_agent_resp = prob_agent.perform_action(agent_job)

print(prob_agent_resp)

data = json.loads(prob_agent_resp)

[
  {
    "ingredients": [
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 0,
        "Probability": 0.0
      },
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 1,
        "Probability": 0.0
      },
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 2,
        "Probability": 0.0
      },
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 3,
        "Probability": 1.0
      },
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 4,
        "Probability": 0.0
      },
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 5,
        "Probability": 0.0
      },
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 6,
        "Probability": 1.0
      },
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 7,
        "Probability": 0.0
      },
      {
        "Topic_Pair": "vintagesxwines",
        "Pair_Index": 8,
        "Probability": 0.0
 

In [16]:
print(len(data[0]["ingredients"]))

48


In [18]:
for ingredient in data[0]["ingredients"]:
    if  ingredient.get("Probability") is not None and ingredient["Probability"] > 0.0:
        print(examples[ingredient["Topic_Pair"]][ingredient["Pair_Index"]], ingredient["Topic_Pair"], ingredient["Probability"])

In [46]:
pprint(df["answer"].iloc[4])

('2 | April 22 | Eastbrook\n'
 '9 | April 10 | Isleton\n'
 '16 | April 17 | Gilmore City\n'
 '23 | April 7 | Manchester')


In [18]:
df["categories"].iloc[3]

'year sent : 1962, 1969, 1976, 1983.\nwriters : Felipe, Karla, Sean, Tracy.\norigins : Bournemouth, Cherbourg, Le Havre, Plymouth.'

In [30]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
from src.utils import accumulate_tools


tool_descriptions = accumulate_tools()
print(tool_descriptions)

[('update_matrix', <function update_matrix at 0x000001BE06698F70>)]
['update_matrix:\n\n    Update a specific cell in one of the probability matrices and then normalize \n    the matrix so each row and column sums to 1 (doubly stochastic).\n    \n    Parameters:\n    -----------\n    matrices : list of numpy.ndarray\n        List of probability matrices\n    matrix_index : int\n        Index of the matrix to update\n    row : int\n        Row index of the cell to update\n    col : int\n        Column index of the cell to update\n    prob_value : float\n        New probability value to set (must be between 0 and 1)\n        \n    Returns:\n    --------\n    list of numpy.ndarray\n        Updated list of matrices with the specified matrix normalized\n    \n\n']


In [31]:

system_prompt = f"""You are an expert in probability theory and are given a sequence a probability updates
                        You are also given a tool to update these matrices and you need to call it sequentially
                        The tools you have access to are:
                        {"".join(tool_descriptions)}
                        \n
                         """

agent_job = """Analyze the updates and return the series of function calls required to make the changes active
            Remember to return a valid JSON of only the function calls of the tool required to make the changes.
"""


In [32]:
from pprint import pprint

import json
prob_agent_2 = Agent(agent_name="prob_agent_2", system_prompt=system_prompt + agent_job, json=True)


new_query =f"The categories are: {categories}\n And here are the probability updates: {prob_agent_resp}"

prob_agent_2_resp = prob_agent_2.perform_action(new_query)

print(prob_agent_2_resp)

#data = json.loads(prob_agent_resp)

[
  {
    "tool_name": "update_matrix",
    "matrix_name": "vintagesxwines",
    "i": 0,
    "j": 3,
    "value": 1.0
  },
  {
    "tool_name": "update_matrix",
    "matrix_name": "vintagesxwines",
    "i": 1,
    "j": 2,
    "value": 1.0
  },
  {
    "tool_name": "update_matrix",
    "matrix_name": "vintagesxwines",
    "i": 2,
    "j": 1,
    "value": 1.0
  },
  {
    "tool_name": "update_matrix",
    "matrix_name": "vintagesxwines",
    "i": 3,
    "j": 0,
    "value": 1.0
  },
  {
    "tool_name": "update_matrix",
    "matrix_name": "vintagesxtypes",
    "i": 0,
    "j": 3,
    "value": 1.0
  },
  {
    "tool_name": "update_matrix",
    "matrix_name": "vintagesxtypes",
    "i": 1,
    "j": 1,
    "value": 1.0
  },
  {
    "tool_name": "update_matrix",
    "matrix_name": "vintagesxtypes",
    "i": 2,
    "j": 2,
    "value": 1.0
  },
  {
    "tool_name": "update_matrix",
    "matrix_name": "vintagesxtypes",
    "i": 3,
    "j": 0,
    "value": 1.0
  },
  {
    "tool_name": "update_m

In [33]:
import json
calls = json.loads(prob_agent_2_resp)

In [34]:
import numpy as np

def update_matrix(updated_matrices, matrix_name, row, col, prob_value):
    """
    Update a specific cell in one of the probability matrices and then normalize 
    the matrix so each row and column sums to 1 (doubly stochastic).
    
    Parameters:
    -----------
    matrices : list of numpy.ndarray
        List of probability matrices
    matrix_index : int
        Index of the matrix to update
    row : int
        Row index of the cell to update
    col : int
        Column index of the cell to update
    prob_value : float
        New probability value to set (must be between 0 and 1)
        
    Returns:
    --------
    list of numpy.ndarray
        Updated list of matrices with the specified matrix normalized
    """
    # Validate probability value is between 0 and 1
    if not 0 <= prob_value <= 1:
        raise ValueError("Probability value must be between 0 and 1")
    
    # Create a copy of the matrices to avoid modifying the original
    
    
    # Update the specified cell
    updated_matrices[matrix_name][row, col] = prob_value
    
    #Apply Sinkhorn-Knopp algorithm for double normalization
   # (make both rows and columns sum to 1)
    matrix = updated_matrices[matrix_name]
    max_iterations = 100
    tolerance = 1e-6
    
    for _ in range(max_iterations):
        # Store matrix before iteration to check for convergence
        prev_matrix = matrix.copy()
        
        # Normalize rows
        row_sums = matrix.sum(axis=1, keepdims=True)
        matrix = matrix / row_sums
        
        # Normalize columns
        col_sums = matrix.sum(axis=0, keepdims=True)
        matrix = matrix / col_sums
        
        # Check convergence
        if np.max(np.abs(matrix - prev_matrix)) < tolerance:
            break
    
    updated_matrices[matrix_name] = matrix
    return updated_matrices


for call in calls:
    fn_ = call["tool_name"]
    if fn_ == "update_matrix":
        

        #print(probability_matrices["placementsxroses"])

        updated_matrices = update_matrix(probability_matrices, matrix_name=call["matrix_name"],
                                            row=int(call["i"]), col=int(call["j"]),prob_value=float(call["value"])
                                            )        
        print(updated_matrices)
        print()
        print()
    


{'vintagesxwines': array([[0.17839458, 0.17839458, 0.17839458, 0.46481623],
       [0.27386847, 0.27386847, 0.27386847, 0.17839459],
       [0.27386847, 0.27386847, 0.27386847, 0.17839459],
       [0.27386847, 0.27386847, 0.27386847, 0.17839459]]), 'vintagesxtypes': array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), 'winesxtypes': array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]])}


{'vintagesxwines': array([[0.192708  , 0.192708  , 0.12613596, 0.48844831],
       [0.19905803, 0.19905803, 0.47574784, 0.12613586],
       [0.30411699, 0.30411699, 0.1990581 , 0.19270792],
       [0.30411699, 0.30411699, 0.1990581 , 0.19270792]]), 'vintagesxtypes': array([[0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25],
       [0.25, 0.25, 0.25, 0.25]]), 'winesxtypes': array([[0.25, 0.25, 0.25, 0.25],

In [36]:
0.15 + 0.15 + 0.15 + 0.55

1.0

In [35]:
pprint(updated_matrices)

{'vintagesxtypes': array([[0.15618339, 0.14297412, 0.14889484, 0.55194797],
       [0.1618281 , 0.54092178, 0.15427613, 0.14297405],
       [0.16852956, 0.1542761 , 0.52829953, 0.14889475],
       [0.51345894, 0.161828  , 0.16852949, 0.15618323]]),
 'vintagesxwines': array([[0.15618339, 0.14889484, 0.14297412, 0.55194797],
       [0.1618281 , 0.15427613, 0.54092178, 0.14297405],
       [0.16852956, 0.52829953, 0.1542761 , 0.14889475],
       [0.51345894, 0.16852949, 0.161828  , 0.15618323]]),
 'winesxtypes': array([[0.15618339, 0.14297412, 0.14889484, 0.55194797],
       [0.1618281 , 0.54092178, 0.15427613, 0.14297405],
       [0.16852956, 0.1542761 , 0.52829953, 0.14889475],
       [0.51345894, 0.161828  , 0.16852949, 0.15618323]])}


In [37]:

system_prompt = f"""You are an expert in probability theory and you are given three matrices. Look at the matrices
                    and finally deduce what category matches what. 

                    Final Answer:\n'
                    'Fill your answers as a table.\n'
                    Fill the following table to show your final answer.\n'
                    'first | correct option from roses | correct option from gardeners\n'
                    'second | correct option from roses | correct option from gardeners\n'
                    'third | correct option from roses | correct option from gardeners\n'
                    'fourth | correct option from roses | correct option from gardeners\n') 
                    Remember, as with all grid-based logic puzzles, no option in any category will ever be used more than once.

                                    
                """

agent_job = f"""Analyze the probability matrix and look for the final solution
                The matrices are: {updated_matrices}
                The categories are: {categories}
"""


In [26]:
categories

['1962, 1969, 1976, 1983.',
 'Felipe, Karla, Sean, Tracy.',
 'Bournemouth, Cherbourg, Le Havre, Plymouth.']

In [27]:
import numpy as np

# Set print options to suppress scientific notation and show 8 decimal places
np.set_printoptions(suppress=True, precision=8)

# Now when you print your arrays, they'll show exact decimal values
print(updated_matrices)


{'year sentxwriters': array([[0.19713932, 0.19297986, 0.        , 0.60987978],
       [0.36974032, 0.06978979, 0.50533101, 0.05513967],
       [0.36333057, 0.35566462, 0.        , 0.28100428],
       [0.06978979, 0.38156573, 0.49466899, 0.05397627]]), 'year sentxorigins': array([[0.18365323, 0.40817354, 0.        , 0.40817354],
       [0.        , 0.21851621, 0.56296707, 0.21851621],
       [0.42663331, 0.18083309, 0.21170063, 0.18083309],
       [0.38971346, 0.19247716, 0.2253323 , 0.19247716]]), 'writersxorigins': array([[0.29454426, 0.        , 0.16938876, 0.53606669],
       [0.52098327, 0.24906788, 0.11439783, 0.11555089],
       [0.18447247, 0.23097529, 0.47739524, 0.10715713],
       [0.        , 0.51995683, 0.23881817, 0.24122529]])}


In [28]:
solution_agent_system_prompt = system_prompt

solution_agent = Agent(agent_name="solution_agent", system_prompt=solution_agent_system_prompt)


solution_agent_resp = solution_agent.perform_action(agent_job)

print(solution_agent_resp)

Okay, I will analyze the probability matrices to deduce the relationships between the categories: years, writers, and origins. The goal is to find the most probable associations between each element of the categories based on the provided matrices.

Here's the strategy:

1.  **Examine the Matrices:** Understand what each matrix represents.
    *   `year sentxwriters`: Probability of a year being associated with a writer.
    *   `year sentxorigins`: Probability of a year being associated with an origin.
    *   `writersxorigins`: Probability of a writer being associated with an origin.

2.  **Find Strongest Probabilities:** Look for the highest probability values in each row of the matrices. These indicate the strongest associations.

3.  **Deduce Relationships:** Combine the information from all three matrices to form a consistent mapping between years, writers, and origins.

4.  **Ensure Uniqueness:** Make sure that each element from each category is used only once.

Let's apply this

In [30]:
pprint(df["answer"].iloc[3])

('1962 | Tracy | Cherbourg\n'
 '1969 | Felipe | Bournemouth\n'
 '1976 | Karla | Plymouth\n'
 '1983 | Sean | Le Havre')
